In [ ]:
# descriptor는 "binding behavior"를 정의하는 객체의 속성이다.
__get__, __set__, __delete__
위의 3가지 메소드가 객체에 정의되어 있다면, 그것을 descriptor라고 한다. 

속성을 접근하는 가장 기본적인 행위는 object의 dictionary를 get, set, delete 하는 것이다. 

예를 들어, a.x 는 선언하면, lookup chain은 다음과 같이 작동한다.
1. a.__dict__['x'] 
2. type(a).__dict__['x']
계속 type(a)의 base class들을 찾기 시작한다. metaclass를 제외하고. 
만약에 조회한 값이 descriptor method 중 하나를 정의한 것이라면, 파이썬은 아마도 기본 행위를 override하고 descriptor method를 호출할 것이다.

descriptor는 매우 강력하고 범용적인 프로토콜이다. 
descriptor는 properties, methods, static methods, class methods, super() 뒤에 숨겨져 있는 메커니즘이다. 

# Descriptor Protocol

descr.__get__(self, obj, type=None) --> value

descr.__set__(self, obj, value) --> None

descr.__delete__(self, obj) --> None

이 3가지가 전부이다. 

만약에 객체가 __get__, __set__ 을 정의한다면 이 객체는 data-descriptor이다. 

In [2]:
descriptor는 메소드 이름을 직접 호출해서 쓸 수 있다. 

obj.d 를 호출할 때 내부적으로 일어나는 일을 적어보면 다음과 같다.
호출의 detail은 obj가 object 인지 아니면 class인지에 따라 다르다. 

SyntaxError: invalid syntax (<ipython-input-2-07e0aeee93bc>, line 1)

In [3]:
# object 인 경우. 
# object.__getattribute__()  tranfrom b.x into 
# b.x ==> type(b).__dict__['x'].__get__(b, type(b))

# full implementation can be found here
# https://hg.python.org/cpython/file/3.5/Objects/object.c

In [4]:
class RevealAccess(object):
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name
        
    def __get__(self, obj, objtype):
        print('retrieve', self.name)
        return self.val
    
    def __set__(self, obj, val):
        print('update', self.name)
        self.val = val
        
class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
    

In [5]:
m = MyClass()
m.x

retrieve var "x"


10

In [6]:
m.x = 20

update var "x"


In [7]:
m.x

retrieve var "x"


20

In [9]:
import types

class Function(object):
    def __get__(self, obj, objtype=None):
        return types.MethodType(self, obj, objtype)

In [11]:
class D(object):
    def f(self, x):
        return x
    
d = D()
D.__dict__['f']

<function __main__.D.f>

In [12]:
D.f

<function __main__.D.f>

In [13]:
d.f

<bound method D.f of <__main__.D object at 0x104a72668>>

In [14]:
import random

class Die:
    def __init__(self, sides = 6):
        self.sides = sides
        
    def __get__(self, instance, owner):
        return int(random.random() * self.sides) + 1

In [15]:
class Game:
    d6 = Die()
    d10 = Die(sides=10)
    d20 = Die(sides=20)

In [16]:
Game.d6

6

In [17]:
Game.d10

10

In [18]:
Game.d20

1

In [19]:
game = Game()
game.d6

6

In [20]:
class CachedValue:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner):
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = sth()
        return instance.__dict__[self.name]
    
    

In [22]:
class Celsius:
    def __init__(self, value=0.0):
        self.value = float(value)
    
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
class Temp:
    celsius = Celsius()
    
Temp.celsius

0.0

In [24]:
class Descriptor:
    def __init__(self):
        self._name = ''
        
    def __get__(self, instance, owner):
        return self._name
    
    def __set__(self, instance, name):
        self._name = name.title()
        
    def __delete__(self, instance):
        del self._name
        
class Person:
    name = Descriptor()

In [25]:
user = Person()
user.name = "john smith"

In [26]:
class Person:
    def __init__(self):
        self._name = ''

    def fget(self):
        return self._name
    
    def fset(self, value):
        self._name = value.title()

    def fdel(self):
        del self._name
        
    name = property(fget, fset, fdel, "property")